# Assignment 1

Deadline: 19.03.2025, 12:00 CET

<Add your name, student-id and emal address>

In [ ]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective, MeanVariance
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS

1.23.0


## 1. Solver horse race

### 1.a)
(3 points)

Generate a Multivariate-Normal random dataset of dimension TxN, T=1000, N=100, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [2]:

# Set the dimensions
T = 256  # Number of time periods
N = 100   # Number of assets

# Generate a random mean vector and covariance matrix for the multivariate normal distribution
mean = np.random.uniform(-0.0005, 0.0005, N)
A = np.random.uniform(0, 0.0113, size=(N, N))
cov = np.dot(A, A.transpose())

# Generate the Multivariate-Normal random dataset
data = np.random.multivariate_normal(mean, cov, size=T)

print('multivariate normal shape =', data.shape)

# Convert the dataset to a DataFrame for easier manipulation
df = pd.DataFrame(data, columns=[f'Asset_{i+1}' for i in range(N)])

# Compute the vector of expected returns (mean returns) from df
scalefactor = 256
expected_return = ExpectedReturn(method='geometric', scalefactor = scalefactor)
q =  expected_return.estimate(df, inplace = False)

# Compute the covariance matrix from df
covariance = Covariance(method='pearson')
P = covariance.estimate(df, inplace=False)

# Display the results
print("Vector of expected returns (q):")
print(q)

print("\nCovariance matrix (P):")
print(P)


multivariate normal shape = (256, 100)
Vector of expected returns (q):
Asset_1     -0.896383
Asset_2     -0.658477
Asset_3     -0.818068
Asset_4     -0.655145
Asset_5     -0.793208
               ...   
Asset_96    -0.658506
Asset_97    -0.811924
Asset_98    -0.822901
Asset_99    -0.734488
Asset_100   -0.797161
Length: 100, dtype: float64

Covariance matrix (P):
            Asset_1   Asset_2   Asset_3   Asset_4   Asset_5   Asset_6  \
Asset_1    0.004502  0.002885  0.003178  0.003588  0.003859  0.003357   
Asset_2    0.002885  0.003468  0.002565  0.003105  0.003230  0.003018   
Asset_3    0.003178  0.002565  0.003945  0.003269  0.003832  0.003366   
Asset_4    0.003588  0.003105  0.003269  0.004793  0.003826  0.003798   
Asset_5    0.003859  0.003230  0.003832  0.003826  0.004985  0.003982   
...             ...       ...       ...       ...       ...       ...   
Asset_96   0.003475  0.002906  0.003227  0.003860  0.003793  0.003644   
Asset_97   0.003656  0.002902  0.003171  0.003609  

In [3]:
# average return over all assets
print(q.mean(axis=0))

-0.7384832250226638


### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the data created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 30 assets is <= 0.3, the sum of assets 31 to 60 is <= 0.4 and the sum of assets 61 to 100 is <= 0.5

In [4]:
# Instantiate the Constraints class
constraints = Constraints(ids = df.columns.tolist())

# Add budget constraint
constraints.add_budget(rhs=1, sense='=')

# Add box constraints (i.e., lower and upper bounds)
constraints.add_box(lower=0.0, upper=0.2)

# Add linear constraints
G = pd.DataFrame(np.zeros((3, N)), columns = constraints.ids)
G.iloc[0, 0:30] = 1
G.iloc[1, 30:60] = 1
G.iloc[2, 60:] = 1
h = pd.Series([0.3, 0.4, 0.5])

constraints.add_linear(G=G, rhs=h, sense='<=')

### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constarints fulfilled? Extract primal resisduals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers that failed for some reason (e.g., unable to install the package or solvers throws an error during execution). 




In [5]:
# Extract the constraints in the format required by the solver
GhAb = constraints.to_GhAb()

# Loop over solvers, instantiate the quadratic program, solve it and store the results
import qpsolvers

print(qpsolvers.available_solvers)
solver_array = ['daqp']
for solver in solver_array:
    mv = MeanVariance(
        covariance=covariance,
        expected_return=expected_return,
        constraints=constraints,
        risk_aversion=1,
        solver_name=solver,
    )
    optimization_data = OptimizationData(return_series=df)
    
    mv.set_objective(optimization_data=optimization_data)

    mv.solve()
    print(mv.results)
    break

['daqp', 'highs', 'osqp', 'qpalm', 'quadprog', 'scs', 'qpax']


SolverNotFound: found solvers ['daqp', 'highs', 'osqp', 'qpalm', 'quadprog', 'scs', 'qpax'] but 'cvxopt' is not one of them; if 'cvxopt' is listed in https://github.com/qpsolvers/qpsolvers#solvers you can run ``pip install qpsolvers[cvxopt]``

Print and visualize the results

In [5]:
#<your code here>

## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [ ]:
# Define class MinVariance
class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        #<your code here>

    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            #<your code here>
            return None
        else:
            return super().solve()


# Create a constraints object with just a budget constraint
#<your code here>

# Instantiate the MinVariance class
#<your code here>

# Prepare the optimization data and prepare the optimization problem
#<your code here>

# Solve the optimization problem and print the weights
#<your code here>